In [2]:
import json
import requests as rq
import pandas as pd
import time

import pymongo
from pymongo import MongoClient
client = MongoClient()
db = client.lol_match_db

list_matches = pd.read_excel('MatchIDList_NoDupes.xlsx',
                      sheetname = 'match_id_list',
                      header = 0,
                      index_col = 0,
                      parse_cols = "B",
                      convert_float = True)
list_matches.head()

""
match_id
2853586095
2807676541
2807350386
2804711583
2795203055


In [8]:
apikey = 'RGAPI-ffa12ae9-fce4-4201-8c73-ed72cc255f0c'
r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/match/' +'2853586095'
           +'?includeTimeline=TRUE&api_key=' + apikey)
print 'https://euw.api.pvp.net/api/lol/euw/v2.2/match/' +'2853586095'+'?includeTimeline=TRUE&api_key=' + apikey
print r.status_code

https://euw.api.pvp.net/api/lol/euw/v2.2/match/2853586095?includeTimeline=TRUE&api_key=RGAPI-ffa12ae9-fce4-4201-8c73-ed72cc255f0c
200


In [24]:
#Execute this second thread concurrently and see what happens

i=200000
while i < 212500:  
    #get all matches for a summoner id
    #WARNING: let's do only one request every two seconds
    r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/match/' +str(list_matches.index[i])
               +'?includeTimeline=TRUE&api_key=' + apikey)
    #check response code
    while (r.status_code <> 200):
        time.sleep(1.5)
        r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/match/' +str(list_matches.index[i])
               +'?includeTimeline=TRUE&api_key=' + apikey)
    
    result = db.match_details_2016.insert_one({"_id": str(list_matches.index[i]), "match_detail":json.loads(r.text)})
    #insert into match_list_2016(contains all matches for Season 2016 for any particular summoner) using
    #the summoner ID as DB Key    
    #increase and wait    
    i = i + 1
    time.sleep(1.25)

In [25]:
print i

212500


In [12]:
print r.status_code

404
